Dashboard building with temperature and abudannce of specific pollinators

import the necessary libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Import Data

In [6]:
x = "pollinator_count_data-final.csv"
y = pd.read_csv("plant_on_campus_frequency.csv", sep = ',')

FileNotFoundError: [Errno 2] No such file or directory: 'plant_on_campus_frequency.csv'

Clean up the imported data

In [ ]:
#want to convert acronyms to full names 
#(hoverfly, honeybee, bumblebee, sweatbee, wasp, leafcutter, fly, beatle, butterfly)

#setting to print all the rows 
pd.set_option('display.max_rows', None)

#observation data for pollinators
df = pd.read_csv(x,sep = ',')

#plants on campus data 
df_frequency = pd.read_csv("plant_on_campus_frequency.csv",sep = ',')

#convert european carder bee ( ECB), mason bee and carpenter bee to the leafcutter bee group (LF)
df['pollinator'] = df['pollinator'].replace(['ECB','MB','CB'],'LB')
#convert No information (NI) and unidentified bee (UB) to the Other pollinator group (OP)
df['pollinator'] = df['pollinator'].replace(['NI','UB'],'OP')

#replacing values within dataframe to favored/changed names exp. Bumble
df['pollinator'] = df['pollinator'].replace(['BUT'],'Butterfly')
df['pollinator'] = df['pollinator'].replace(['W'],'Wasp')
df['pollinator'] = df['pollinator'].replace(['HF'],'Hover Fly')
df['pollinator'] = df['pollinator'].replace(['BVO','BS','BMI','BME','BU','BI','BFL','BF','BB'],'Bumble')
df['pollinator'] = df['pollinator'].replace(['SB'],'Sweat')
df['pollinator'] = df['pollinator'].replace(['HB'],'Honey')
df['pollinator'] = df['pollinator'].replace(['LB'],'Leaf Cutter')



#using where statement to seperate OP to beetles and flies 

# beetle 
df['pollinator'] = np.where((df['pollinator'] == 'OP') & (df['insect_order'] == 'coleoptera'), 'Beetle', df['pollinator'])
#print(df['pollinator'])

# flies 
df['pollinator'] =  np.where((df['pollinator'] == 'OP') & (df['insect_order'] == 'diptera'),'Fly',df['pollinator'] )
#print(df[['pollinator','insect_order']])

#droping all other pollinator from the data because not interested in other pollinators in data analysis
df = df.drop(df[df['pollinator'] == 'OP'].index)
#print(df[['pollinator','insect_order']])

In [ ]:

def create_unique_C(x):



	df['sample_date'] = pd.to_datetime(df['sample_date'])

	#find the count of morphogroup in the csv 
	pol_count = df.groupby(['pollinator'])['pollinator'].count()
	print('the pollinator count is', pol_count)
	print('the # of pollinators is', len(df))
	print((pol_count/len(df))*100)


	#percentage of plants in libary garden 

	df_copy = df_frequency.loc[df_frequency['location_number'].str.contains('L',flags=re.I,regex=True)]
	library_counts = df_copy.groupby(['genus'])['genus'].count()
	library_counts = library_counts.reset_index(name = 'count')

	for index,row in library_counts.iterrows():
		row['count'] = (row['count']/11)*100
		#print(row['genus'],row['count'])
	#print('spagetti')

	#percentage of plants in community garden 
	
	df_copy = df_frequency.loc[df_frequency['location_number'].str.contains('C',flags=re.I,regex=True)]
	community_counts = df_copy.groupby(['genus'])['genus'].count()
	community_counts = community_counts.reset_index(name = 'count')

	for index,row in community_counts.iterrows():
		row['count'] = (row['count']/11)*100
		#print(row['genus'],row['count'])
	#print('spagetti')

	#percentage of plants in student union garden
	df_copy = df_frequency.loc[df_frequency['location_number'].str.contains('S',flags=re.I,regex=True)]
	student_counts = df_copy.groupby(['genus'])['genus'].count()
	student_counts = student_counts.reset_index(name = 'count')

	for index,row in student_counts.iterrows():
		row['count'] = (row['count']/11)*100
		#print(row['genus'],row['count'])
	#print('spagetti')

	#percent visited in observation
	percent_visit = df.groupby(['plant_genus'])['plant_genus'].count()
	percent_visit = percent_visit.reset_index(name = 'count')

	temp_S = len(df.index)
	#print(temp_S)

	#for index,row in percent_visit.iterrows():
		#row['count'] = (row['count']/temp_S)*100
		#print(row['plant_genus'],row['count'])

	#observed data 
	#create unique values of genus with count as a dataframe
	#for each location number, what unique plant genus are in the observed data

	temp_counts = df.drop_duplicates(subset = ['location_number','plant_genus'])
	temp_counts = temp_counts.groupby(['plant_genus'])['plant_genus'].count()
	temp_counts = temp_counts.reset_index(name = 'counts')




	
	temp_list = list()


	#renaming the column
	temp_counts.rename({'genus': 'plant_genus'}, axis=1)

	#merging the two columns together into one dataframe 
	#idea --> using merges in SQL
	# find a way to still display the number
	#currently not in use 
	temp = percent_counts.merge(pollinator_counts,
								how = 'right',
								left_on = ['genus','count'], # genus and count of plants on campus per plot
								right_on = ['plant_genus','counts']) # genus and count in observed data
	
	#for index,row in temp.iterrows():
	#	print(row['genus'],row['count'],row['plant_genus'],row['counts'])


	
	#print(percent_counts)
	#print(temp_counts)





	return df


In [ ]:
df['sample_date'] = pd.to_datetime(df['sample_date'])

#find the count of morphogroup in the csv 
pol_count = df.groupby(['pollinator'])['pollinator'].count()
print('the pollinator count is', pol_count)
print('the # of pollinators is', len(df))
print((pol_count/len(df))*100)

Define a function which would give the average temperature 

In [2]:
def line_temp_plot(data_set):

	#print(data_set.columns)

	data_set['month'] = data_set['sample_date'].dt.month
	data_set['Week'] = data_set['sample_date'].dt.week
	data_set['day_of_year'] = data_set['sample_date'].dt.dayofweek


	#TEST to find missing temperatures in data 
	#Result : found missing time data for C0: E and C10: A --> both have no data for count in pollinator
	#must hardcode into data as no pollinator 
	count_test = data_set.groupby(['location_number','time_code'])['location_number'].count()
	count_test = count_test.reset_index(name='counts')
	#print(count_test)

	#results 
	# week = 20 , air_temp = 15, time_code = 'E', location_number = 'CO', counts = 0
	# week = 35 , air_temp = 23, time_code = 'A', location_number = 'C10', counts = 0
	#print(data_set.columns)
	num_plot_week = data_set.groupby(['Week','air_temp','time_code','location_number'])['air_temp'].count()
	num_plot_week = num_plot_week.reset_index(name = 'counts')
	pd.set_option('display.max_rows', None)
	
	num_plot_week.loc[len(num_plot_week)] = [20,15,'E','C0',0]
	num_plot_week.loc[len(num_plot_week)] = [35,23,'A','C10',0]
	print('test_run','\n',num_plot_week)


	num_plot_week = num_plot_week.groupby(['Week'])['air_temp'].mean()
	num_plot_week = num_plot_week.reset_index(name = 'avg_air_temp')
	data_set = pd.merge(data_set, num_plot_week)
	
	#print(data_set)
	return data_set

In [ ]:
def average_per_week(data_set):
	#get a column of month number of year
	data_set['month'] = data_set['sample_date'].dt.month
	#get a column of  the week of year starting at 0
	data_set['Week'] = data_set['sample_date'].dt.week -20
	#get a column of  the day of the year 
	data_set['day_of_year'] = data_set['sample_date'].dt.dayofweek
	#Get a column of the start of the week of all datetime objects
	data_set['datetime_monday_week'] = data_set['sample_date'].dt.to_period('W').dt.start_time
	
	#Test Case :1
	#print("Test Case : 1", "\n",data_set['datetime_monday_week'])

	#find the total number of pollinators according to location number and time code
	data_set['total_counts'] = data_set.groupby(['location_number','time_code'])['location_number'].transform('count')

	#Test Case :2
	#print("Test Case : 2", "\n",data_set)

	#creating average temerapture for all plots 
	data_set = line_temp_plot(data_set)

	#getting the average pollinator mean 
	data_set['pol_mean'] = data_set.groupby(['datetime_monday_week'])['total_counts'].transform('mean')

	#
	print(data_set)


	#finding the unique average air temperature by using .last()
	data_set = data_set.groupby(['datetime_monday_week','pol_mean'])['avg_air_temp'].last()
	data_set = data_set.reset_index(name = 'avg_temp')
	print(data_set)

	#test if statistically significant with linear regression 
	#significant = scatter( data_set['avg_temp'], data_set['pol_mean'])

	#plotting the graph

	fig, ax1 = plt.subplots(figsize=(10, 8))

	ax1.plot(data_set['datetime_monday_week'], data_set['pol_mean'], color = 'red')
	ax2 = ax1.twinx()
	ax2.plot(data_set['datetime_monday_week'],data_set['avg_temp'], color = 'blue')

	#changing the scale of the x axis of the graph
	ax1.set_yticks(np.linspace(5, 95, 10))
	ax1.set_ylim(5,95)
	ax2.set_yticks(np.linspace(12, 30, 10))
	
	#ax2.legend(loc=0)
	#plt.legend(bbox_to_anchor = (0.75, 1.15), ncol = 2)

	#ax2.set_ylim(ax1.get_ylim())
	ax2.grid(True)

	ax1.set_xlabel('Week', fontsize = 17)
	
	ax1.set_ylabel('Pollinator Average Count', fontsize = 17)
	ax1.tick_params(axis = "y", labelcolor = 'red', labelsize = 15)
	ax1.tick_params(axis = 'x',labelsize = 15 )
	ax1.xaxis.grid()
	ax1.yaxis.grid()

	ax2.set_ylabel("Average Air Temp (C°)", fontsize = 17)
	ax2.tick_params(axis = 'y', labelcolor = 'blue',labelsize = 15)

	#ax1.legend(loc='best', fontsize = '80')

	fig.suptitle('Average Pollinator Count and Temperature Across 14 Weeks', fontsize = 20)
	fig.autofmt_xdate()

	
	plt.savefig('pollinator_avg_week.png',dpi = 500)

	plt.show()